# Huffman Coding – Homework

This notebook is your homework assignment on the topic of **Huffman coding**.




## Submission Instructions

Your final work must be submitted on **your personal GitHub account**, in a **public repository** dedicated to this assignment.

Your repository must include:

- the completed and cleanly executed notebook,
- any additional files you created for testing or analysis,
- a short `README.md` explaining how to run your notebook.

The deadline for submission is **December 15** (included).  
Any repository created or updated after this date will be considered late.



## Assignment Overview

Start by reading the wiki page on [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) to understand the basic concepts.

**Objectives**:

- Compute character frequencies in a text.
- Build a Huffman tree using a min-heap.
- Derive binary codes from a Huffman tree.
- Implement encoding and decoding functions.

Read each text cell carefully, then complete the corresponding code cells.
You can run the test cells to check your work step by step.

## Exercise 1 – Character frequencies

We first need a function that counts how often each character appears in a text.

- The function should **ignore case**: treat `"A"` and `"a"` as the same character.
- It should return a **dictionary** whose keys are characters and whose values are their frequencies.

> Example:  
> `frequency("abA!")` could return `{ "a": 2, "b": 1, "!": 1 }` (the order of keys does not matter).


In [1]:
def frequency(text: str) -> dict[str, int]:
    """Return a dictionary mapping each character in `text` to its frequency.
    
    The function must:
    - convert the text to lowercase;
    - count how many times each character appears.
    """

    text = text.lower()
    freq_dict = {}
    for letter in text:
        freq_dict[letter] = freq_dict.get(letter, 0) + 1
    return freq_dict
    pass
frequency("abA!")

{'a': 2, 'b': 1, '!': 1}

In [2]:
# === Tests for Exercise 1 ===

assert frequency("") == {}
assert frequency("aaa") == {"a": 3}
assert frequency("abA!") == {"a": 2, "b": 1, "!": 1}

print("Exercise 1 seems OK (at least for these simple tests).")


Exercise 1 seems OK (at least for these simple tests).


## Exercise 2 – A node for the Huffman tree

We represent a Huffman tree using a `Node` class.

Each node must store:

- `characters`: a string containing all characters that appear in the subtree rooted at this node;
- `frequency`: the sum of the frequencies of these characters;
- `left`: the left child (another `Node`, or `None` for a leaf);
- `right`: the right child (another `Node`, or `None` for a leaf).

We will also define the method `__lt__` so that nodes can be compared by frequency when we use them in a **min-heap**.


In [3]:
class Node:
    def __init__(self, characters: str, frequency: int):
        """Create a new Huffman node.
        - 'characters' is the concatenation of all characters in this subtree.
        - 'frequency' is the total frequency of these characters.
        """
        self.characters = characters
        self.frequency = frequency
        self.left = None
        self.right = None   

    def __lt__(self, other: "Node") -> bool:
        """Compare two nodes by frequency.
        This is needed so that we can use `heapq` on lists of nodes.
        """
        return self.frequency < other.frequency

In [4]:
# === Tests for Exercise 2 ===

n1 = Node("a", 5)
n2 = Node("b", 3)

# Check attributes exist
assert hasattr(n1, "characters")
assert hasattr(n1, "frequency")
assert hasattr(n1, "left")
assert hasattr(n1, "right")

# Check initial values
assert n1.characters == "a"
assert n1.frequency == 5
assert n1.left is None and n1.right is None

# Check comparison works (this must not crash)
nodes = [n1, n2]
from heapq import heapify
heapify(nodes)

print("Exercise 2 seems OK (basic tests passed).")


Exercise 2 seems OK (basic tests passed).


## Exercise 3 – Building the Huffman tree with a min-heap

We now build the Huffman tree from a dictionary of character frequencies.

**Algorithm (Huffman’s algorithm):**

1. Create a leaf `Node` for each character and push all these nodes into a **min-heap**.
2. While the heap contains more than one node:
   1. Pop the two nodes with the smallest frequency.
   2. Create a new internal node whose:
      - `left` child is the first node,
      - `right` child is the second node,
      - `characters` is the concatenation of the children’s `characters`,
      - `frequency` is the sum of the children’s `frequency`.
   3. Push this new node back into the heap.
3. The last remaining node in the heap is the **root of the Huffman tree**.


In [5]:
from heapq import heappush, heappop, heapify 

def huffman_tree(freqs: dict[str, int]) -> Node:
    """Build the Huffman tree from a dictionary of frequencies.
    'freqs' maps characters to their frequencies.
    Return the root node of the Huffman tree.
    """
    if not freqs:  
        return None
    
    if len(freqs) == 1:
        char, freq = next(iter(freqs.items()))
        return Node(char, freq)
    
    heap = [Node(char, freq) for char, freq in freqs.items()]
    heapify(heap) 
    
    while len(heap) > 1:
        left = heappop(heap)
        right = heappop(heap)
        parent = Node(left.characters + right.characters, 
                     left.frequency + right.frequency)
        parent.left, parent.right = left, right
        heappush(heap, parent)
    
    return heap[0]

In [6]:
# === Tests for Exercise 3 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# Basic structural checks
assert isinstance(root, Node)
assert set(root.characters) == set(freqs.keys())
assert root.frequency == sum(freqs.values())

print("Exercise 3 seems OK (basic tests passed).")


Exercise 3 seems OK (basic tests passed).


## Exercise 4 – Code of a single character

We now want a function that returns the **binary code** of a given character,
based on the Huffman tree.

We assume that:

- a left edge corresponds to the bit `'0'`,
- a right edge corresponds to the bit `'1'`.

**Specification:**

- If `char` does **not** appear in the tree, raise a `ValueError`.
- Otherwise, return a string of `'0'` and `'1'` representing the path from the root to the leaf that contains `char`.
- You may implement this function **recursively**.


In [7]:
def get_code(tree: Node, char: str) -> str:
    """Return the Huffman code of 'char' in the given tree.
    Raise a ValueError if 'char' is not present in the tree.
    """
    def traverse(node, target_char, code_so_far=""):
        if node is None:
            return None
        
        if node.left is None and node.right is None:
            if target_char in node.characters:
                return code_so_far
            return None
        
        if target_char not in node.characters:
            return None
        
        if node.left:
            left_result = traverse(node.left, target_char, code_so_far + "0")
            if left_result is not None:
                return left_result
        
        if node.right:
            right_result = traverse(node.right, target_char, code_so_far + "1")
            if right_result is not None:
                return right_result
        
        return None
    
    result_code = traverse(tree, char)
    
    if result_code is None:
        raise ValueError(f"Character '{char}' not found in tree")
    
    return result_code


In [8]:
# === Tests for Exercise 4 ===

freqs = frequency("aaaabbc")
root = huffman_tree(freqs)

code_a = get_code(root, "a")
code_b = get_code(root, "b")
code_c = get_code(root, "c")

# Codes must be non-empty and different for different characters
assert isinstance(code_a, str) and len(code_a) > 0
assert isinstance(code_b, str) and len(code_b) > 0
assert isinstance(code_c, str) and len(code_c) > 0

assert code_a != code_b or code_a != code_c or code_b != code_c

print("Exercise 4 seems OK (basic tests passed).")


Exercise 4 seems OK (basic tests passed).


## Exercise 5 – Listing all codes

We now want to **print all Huffman codes** for all characters in the tree.

Complete the function `show_all_codes(tree)` so that it prints,
for each character in the tree, a line of the form:

```text
Character: x Code: 0101
```

You should reuse your function `get_code` from the previous exercise.


In [9]:
def show_all_codes(tree: Node) -> None:
    """Print the Huffman code of each character in the tree."""
    if tree is None:
        print("Tree is empty")
        return
    
    unique_chars = sorted(set(tree.characters))
    
    for character in unique_chars:
        huffman_code = get_code(tree, character)
        print(f"Character: {character} Code: {huffman_code}")

In [10]:
# === Manual test for Exercise 5 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# This should print one line per distinct character
show_all_codes(root)


Character:   Code: 101
Character: a Code: 1111
Character: c Code: 01111
Character: d Code: 01011
Character: e Code: 1110
Character: f Code: 1101
Character: g Code: 10001
Character: h Code: 0100
Character: i Code: 1001
Character: l Code: 01101
Character: m Code: 0011
Character: n Code: 000
Character: o Code: 11001
Character: p Code: 10000
Character: r Code: 01100
Character: s Code: 0010
Character: t Code: 01010
Character: u Code: 11000
Character: x Code: 01110


## Exercise 6 – Encoding and decoding

We now implement the two main operations:

- `huffman_encode(text, tree)` returns a **string of bits** (a string of `'0'` and `'1'`),
- `huffman_decode(encoded_text, tree)` returns the original text.

### 6.1 – Encoding

To encode a text:

1. For each character `c` in `text`, look up its code with `get_code(tree, c)`.
2. Concatenate all these codes into a single string.

If a character of `text` does not appear in the tree, you may raise a `ValueError`.


In [11]:
def huffman_encode(text: str, tree: Node) -> str:
    """Return the Huffman encoding of 'text' using the given tree."""
    text = text.lower()
    
    encoded = ""
    for char in text:
        code = get_code(tree, char)
        encoded += code
    
    return encoded

### 6.2 – Decoding

To decode a string of bits:

1. Start at the **root** of the tree.
2. For each bit in the encoded string:
   - if the bit is `'0'`, go to the **left** child;
   - if the bit is `'1'`, go to the **right** child.
3. Each time you reach a **leaf** (a node whose `left` and `right` are `None`):
   - append its character to the decoded text,
   - go back to the root.

At the end, you should have reconstructed the original text.


In [12]:
def huffman_decode(encoded_text: str, tree: Node) -> str:
    """Decode 'encoded_text' using the given Huffman tree."""
    if not encoded_text or tree is None:
        return ""
    
    decoded_text = ""
    current_node = tree
    
    for bit in encoded_text:
        # Only process valid bits
        if bit not in '01':
            continue  # Skip invalid characters
        
        if bit == '0':
            current_node = current_node.left
        else:  # bit == '1' (guaranteed by the check above)
            current_node = current_node.right
        
        if current_node.left is None and current_node.right is None:
            decoded_text += current_node.characters
            current_node = tree
    
    return decoded_text

In [13]:
# === Tests for Exercise 6 ===

text = "this is an example for huffman encoding"
freqs = frequency(text)
root = huffman_tree(freqs)

encoded = huffman_encode(text, root)
decoded = huffman_decode(encoded, root)

print("Encoded text:", encoded[:64] + ("..." if len(encoded) > 64 else ""))
print("Decoded text:", decoded)

assert decoded == text.lower(), "Decoded text does not match original (lowercased)."

print("Exercise 6 seems OK (basic round-trip test passed).")


Encoded text: 0101001001001001010110010010101111100010111100111011110011100000...
Decoded text: this is an example for huffman encoding
Exercise 6 seems OK (basic round-trip test passed).


## Exercise 7 – A “universal” Huffman tree for English

So far, all Huffman trees you built were specialised to a specific text.  
In this exercise, you will construct a **general-purpose Huffman tree for English**.

For that purpose, we provide you with a dictionary `english_frequencies` that gives the approximate frequency of each character in English texts.

In [14]:
freqs_english = {
    " ": 18.0,
    "e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95,
    "s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88,
    "c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03,
    "p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11,
    "j": 0.10, "z": 0.07
}


Start by building a Huffman tree from this dictionary and test it by encoding and decoding the following text:

```text
"Huffman coding is a data compression algorithm." 
```

Make sure that the decoded text matches the original text!


In [15]:
int_freqs = {char: int(freq * 100) for char, freq in freqs_english.items()}
english_tree = huffman_tree(int_freqs)

Compare the length of the encoded text with the length of the original text (in bits).You can assume that each character in the original text is represented using 8 bits (1 byte) using [ASCII encoding](https://en.wikipedia.org/wiki/ASCII).

In [16]:
test_text = "huffman coding is a data compression algorithm"
encoded = huffman_encode(test_text, english_tree)
decoded = huffman_decode(encoded, english_tree)

print("Test text:", test_text)
print("Encoded length:", len(encoded), "bits")
print("Original length:", len(test_text) * 8, "bits")
print("Compression ratio:", round((1 - len(encoded)/(len(test_text)*8))*100, 1), "%")
print("Decoded matches original:", decoded == test_text.lower())

Test text: huffman coding is a data compression algorithm
Encoded length: 199 bits
Original length: 368 bits
Compression ratio: 45.9 %
Decoded matches original: True
